In [ ]:
import pandas as pd
import re
anime = pd.read_csv("anime-dataset-2023.csv")
anime.head()

In [ ]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

anime['clean_title'] = anime['English name'].apply(clean_title)


anime_cleaned = anime.dropna(subset=['Name', 'clean_title', 'Score', 'anime_id'])


print(anime_cleaned[['Name', 'clean_title']].head())

                              Name             clean_title
0                     Cowboy Bebop            Cowboy Bebop
1  Cowboy Bebop: Tengoku no Tobira  Cowboy Bebop The Movie
2                           Trigun                  Trigun
3               Witch Hunter Robin      Witch Hunter Robin
4                   Bouken Ou Beet  Beet the Vandel Buster


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(anime["clean_title"])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to search for anime by title and return the English name
def search(name):
    # Filter anime DataFrame to find matching titles
    results = anime[anime['Name'].str.contains(name, case=False, na=False)]

    if results.empty:
        print("No results found.")
        return pd.DataFrame()  # Return an empty DataFrame if no matches are found

    # Ensure results are sorted or handled as needed
    return results[['anime_id', 'English name']]

In [ ]:
import ipywidgets as widgets
from IPython.display import display

anime_input = widgets.Text(
    value='',
    description='Anime Name:',
    disabled=False
)

anime_list = widgets.Output()

def on_type(data):
    with anime_list:
        anime_list.clear_output()
        name = data["new"]
        if len(name) > 2:  # Check if the input length is sufficient
            results = search(name)
            if not results.empty:
                # Display the results with English names
                display(results)

anime_input.observe(on_type, names='value')

display(anime_input, anime_list)

Text(value='', description='Anime Name:')

Output()

In [ ]:

ratings = pd.read_csv("rating.csv")
animeID = 21
animes = anime[anime["anime_id"] == animeID]
ratings.dtypes


,0
user_id,int64
anime_id,int64
rating,int64


In [ ]:
similar_users = ratings[(ratings["anime_id"] == animeID) & (ratings["rating"] > 7)]["user_id"].unique()
similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) & (ratings["rating"] > 7)]["anime_id"]
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs = similar_user_recs[similar_user_recs > 0.1]



In [ ]:
all_users = ratings[(ratings["anime_id"].isin(similar_user_recs.index)) & (ratings["rating"] > 7)]
all_user_recs = all_users["anime_id"].value_counts() / len(all_users["user_id"].unique())

In [ ]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]



In [ ]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]


rec_percentages = rec_percentages.sort_values("score", ascending=False)

rec_percentages.head(10).merge(anime, left_index=True, right_on="anime_id")



,similar,all,score,anime_id,Name,English name,Other name,Score,Genres,Synopsis,...,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL,clean_title


In [ ]:
def find_similar_anime(anime_id):
    similar_users = ratings[(ratings["anime_id"] == anime_id) & (ratings["rating"] > 4)]["user_id"].unique()
    similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) & (ratings["rating"] > 4)]["anime_id"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["anime_id"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["anime_id"].value_counts() / len(all_users["user_id"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("similar", ascending=False)
    # Return columns that exist in the DataFrame
    return rec_percentages.head(10).merge(anime, left_index=True, right_on="anime_id")


In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='',
    description='Anime :',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        name = data["new"]
        if len(name) > 5:
            results = search(name)
            animeID = results.iloc[0]["anime_id"]
            display(find_similar_anime(animeID))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)


Text(value='', description='Anime :')

Output()